In [1]:
pip install xmltodict

In [2]:
import requests
import xmltodict
import pandas as pd
from math import atan

import pprint

# Helper Functions

In [3]:
def fahrenheitToCelsius(faren):
  return float((faren - 32) * (5/9))

In [4]:
def calcWetBulb(temp, rh):
  term1 = temp*atan(0.151977 * (rh + 8.313659)**(1/2))
  term2 = atan(temp + rh) - atan(rh - 1.676331)
  term3 = 0.00391838 *(rh)**(3/2) * atan(0.023101 * rh) - 4.686035

  return term1 + term2 + term3

# Main Functions

In [13]:
# Coordinates need to be in the form of: "lat1,long2 lat2,long2 lat3,long3"
# Example: "38.99,-77.02 39.70,-104.80 47.6,-122.30"

def multiGrab(coordinates):
  count = coordinates.count(" ") + 1

  seperate_coords = coordinates.split(" ")
  coords = []
  for c in seperate_coords:
    coords.append([float(x) for x in c.split(",")])

  URL = "http://graphical.weather.gov/xml/sample_products/browser_interface/ndfdXMLclient.php"
  # URL of NOAA servers

  # Grab current UTC time
  import time
  epoch_time = time.time()
  UTC_time = time.localtime(epoch_time)

  # Query setup for REST GET request
  query = {"listLatLon": coordinates, # format: "lat1,long1 lat2,long2 lat3,long3" Note the spaces!
         "product": "time-series",
         "Unit": "e",
         "temp": "temp",
         "rh" : "rh",
         "wspd" : "wspd",
         "begin": f"{UTC_time.tm_year}-{UTC_time.tm_mon}-{UTC_time.tm_mday}" # Year, Month, Day
         }

  # GET data from servers
  response = requests.get(URL, params=query)
  dict_data = xmltodict.parse(response.content) # Convert XML format to dict (easier to work with)

  # Navigate to relevant values in dict
  ## head = dict_data['dwml']['data']
  times = dict_data['dwml']['data']['time-layout']
  parameters = dict_data['dwml']['data']['parameters']

  return_values = []
  
  for i in range(count):
    lat, longo = coords[i]
    humidity = list(map(int, parameters[i]['humidity']['value']))
    ftemp = list(map(float, parameters[i]['temperature']['value']))
    ctemp = list(map(fahrenheitToCelsius, ftemp))
    time = times[i]['start-valid-time']
    wetBulb = list(map(calcWetBulb, ctemp, humidity))

    data = {
        'Time': time,
        'Temperature': ftemp,
        'Relative Humidity': humidity,
        'Wet Bulb Temperature': wetBulb,
        }
    
    df = pd.DataFrame(data=data)
    return_values.append(((lat, longo), df))

  return return_values

In [11]:
# Zipcode should be in form of "02464"
# Will grab relevent data from that specific zipcode
def singleGrab(zipcode):
  URL = "http://graphical.weather.gov/xml/sample_products/browser_interface/ndfdXMLclient.php"

  import time
  epoch_time = time.time()
  UTC_time = time.localtime(epoch_time)

  query = {"zipCodeList": zipcode,
         "product": "time-series",
         "Unit": "e",
         "temp": "temp",
         "rh" : "rh",
         "wspd" : "wspd",
         "begin": f"{UTC_time.tm_year}-{UTC_time.tm_mon}-{UTC_time.tm_mday}" # Year, Month, Day
         }
  
  response = requests.get(URL, params=query)
  dict_data = xmltodict.parse(response.content)

  # dict_data['dwml']['data']
  head = dict_data['dwml']['data']
  parameters = dict_data['dwml']['data']['parameters']
  lat = float(dict_data['dwml']['data']['location']['point']['@latitude'])
  longo = float(dict_data['dwml']['data']['location']['point']['@longitude'])
  humidity = dict_data['dwml']['data']['parameters']['humidity']['value']
  temp = dict_data['dwml']['data']['parameters']['temperature']['value']
  time = dict_data['dwml']['data']['time-layout']['start-valid-time']

  data = {'Time': time,
          'Temperature': temp,
          'Relative Humidity': humidity}

  df = pd.DataFrame(data=data)
  # print("Humidty Count:", len(humidity))
  # print("Temp Count:", len(temp))
  # print("Time Count:", len(time))

  # print(f"Zipcode: {zipcode}")
  # print(f"Lat: {lat}, Long: {longo}")
  # print(df)

  return((lat, longo), df)

  # Add equation for cold bulb temp

# Test Code:

In [16]:
coor = "38.99,-77.02 39.70,-104.80 47.6,-122.30"

multiGrab(coor)

[                         Time  ...  Wet Bulb Temperature
 0   2021-10-03T23:00:00-04:00  ...             19.643296
 1   2021-10-04T02:00:00-04:00  ...             18.941183
 2   2021-10-04T05:00:00-04:00  ...             18.713237
 3   2021-10-04T08:00:00-04:00  ...             18.590509
 4   2021-10-04T11:00:00-04:00  ...             20.596157
 5   2021-10-04T14:00:00-04:00  ...             21.498680
 6   2021-10-04T17:00:00-04:00  ...             21.078705
 7   2021-10-04T20:00:00-04:00  ...             20.411396
 8   2021-10-04T23:00:00-04:00  ...             20.010196
 9   2021-10-05T02:00:00-04:00  ...             19.450708
 10  2021-10-05T05:00:00-04:00  ...             18.891219
 11  2021-10-05T08:00:00-04:00  ...             18.891219
 12  2021-10-05T11:00:00-04:00  ...             20.700220
 13  2021-10-05T14:00:00-04:00  ...             21.776862
 14  2021-10-05T17:00:00-04:00  ...             21.249574
 15  2021-10-05T20:00:00-04:00  ...             20.370133
 16  2021-10-0

In [8]:
fahrenheitToCelsius(100)

37.77777777777778

In [9]:
calcWetBulb(37, 99)

36.970520098182874

In [10]:
singleGrab("02464")

((42.31, -71.22),                          Time Temperature Relative Humidity
 0   2021-10-03T23:00:00-04:00          59                96
 1   2021-10-04T02:00:00-04:00          58               100
 2   2021-10-04T05:00:00-04:00          58               100
 3   2021-10-04T08:00:00-04:00          57                96
 4   2021-10-04T11:00:00-04:00          57                93
 5   2021-10-04T14:00:00-04:00          58                93
 6   2021-10-04T17:00:00-04:00          57                96
 7   2021-10-04T20:00:00-04:00          57                97
 8   2021-10-04T23:00:00-04:00          56               100
 9   2021-10-05T02:00:00-04:00          56               100
 10  2021-10-05T05:00:00-04:00          56                96
 11  2021-10-05T08:00:00-04:00          56                93
 12  2021-10-05T11:00:00-04:00          58                97
 13  2021-10-05T14:00:00-04:00          59                90
 14  2021-10-05T17:00:00-04:00          59                90
 15  20

Credits:
*   [Graphical Weather Gov Documentation](https://graphical.weather.gov/xml/rest.php)
*   [Stack Overflow: Download Weather Data by Zipcode](https://stackoverflow.com/questions/26760616/download-weather-data-by-zip-code-tabulate-area-in-r)

